In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

/Users/kimberlybraud/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# 1. Create a function that takes in three arguments; Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def etl():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv('ratings.csv')

    # 3. Open the read the Wikipedia data JSON file.
    with open('wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

In [3]:
# 6 Create the path to your file directory and variables for the three files. 
file_dir = 'C://Users/kimberlyrbraud/vdata/mod_8/Mod_8_V2/08-ETL/Resources/'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [4]:
# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = etl()

In [5]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [6]:
pd.set_option("display.max_columns", None)
wiki_movies_df.sample()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,Narrated by,Music by,Cinematography,Edited by,Productioncompany,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Written by,Genre,Theme music composer,Country of origin,Original language(s),Producer(s),Editor(s),Production company(s),Original network,Original release,Productioncompanies,Executive producer(s),Production location(s),Distributor,Picture format,Audio format,Voices of,Followed by,Composer(s),Created by,Also known as,Opening theme,No. of episodes,Preceded by,Author,Publisher,Publication date,Media type,Pages,ISBN,OCLC,LC Class,Cover artist,Series,Set in,Adaptation by,Suggested by,Biographical data,Born,Died,Resting place,Occupation,Years active,Spouse(s),Children,Parent(s),Genres,Instruments,Labels,Website,Traditional,Mandarin,Type,Industry,Fate,Founded,Founder,Headquarters,Parent,Released,Recorded,Venue,Length,Label,Director,Producer,Area,Coordinates,Status,Opening date,Closing date,Replaced,Replaced by,Name,Attraction type,Music,Duration,Simplified Chinese,Traditional Chinese,Hanyu Pinyin,Literal meaning,Transcriptions,Bopomofo,Gwoyeu Romatzyh,Wade–Giles,IPA,Yale Romanization,Jyutping,Hokkien POJ,Animation by,Color process,Engine(s),Genre(s),Actor control,Production company,Release(s),Format(s),Simplified,Characters,Date premiered,Place premiered,Setting,Original language,Subject,Published,Dewey Decimal,Text,Illustrator,Original title,Published in English,French,Developed by,Ending theme,No. of seasons,Nationality,Portrayed by,Alias,Species,Gender,Family,Alma mater,Camera setup,Novel(s),Comics,Film(s),Screen story by,Hangul,Revised Romanization,McCune–Reischauer,Developer(s),Publisher(s),Designer(s),Programmer(s),Artist(s),Writer(s),Engine,Platform(s),Release,Mode(s),Original work,Television series,Japanese,Hepburn,Literally,Cantonese,Full name,Height,Seasons,Chinese,Other names,Relatives,Yiddish,Formerly,Key people,Total assets,Owner,Number of employees,Divisions,Subsidiaries,Arabic,Romanized,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
5242,https://en.wikipedia.org/wiki/Case_39,2010.0,https://www.imdb.com/title/tt0795351/,Case 39,Christian Alvart,"[Lisa Bruce, Steve Golin, Alix Madigan, Kevin ...",NaN,NaN,NaN,"[Renée Zellweger, Jodelle Ferland, Ian McShane...",NaN,Michl Britsch,Hagen Bodanski,Mark Goldblatt,Paramount Vantage,Paramount Pictures,"[August 13, 2009, (, 2009-08-13, ), (New Zeala...",109 minutes,"[United States, Canada]",English,$26 million,$28.2 million,Ray Wright,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,Narrated by,Music by,Cinematography,Edited by,Productioncompany,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Written by,Genre,Theme music composer,Country of origin,Original language(s),Producer(s),Editor(s),Production company(s),Original network,Original release,Productioncompanies,Executive producer(s),Production location(s),Distributor,Picture format,Audio format,Voices of,Followed by,Composer(s),Created by,Also known as,Opening theme,No. of episodes,Preceded by,Author,Publisher,Publication date,Media type,Pages,ISBN,OCLC,LC Class,Cover artist,Series,Set in,Adaptation by,Suggested by,Biographical data,Born,Died,Resting place,Occupation,Years active,Spouse(s),Children,Parent(s),Genres,Instruments,Labels,Website,Traditional,Mandarin,Type,Industry,Fate,Founded,Founder,Headquarters,Parent,Released,Recorded,Venue,Length,Label,Director,Producer,Area,Coordinates,Status,Opening date,Closing date,Replaced,Replaced by,Name,Attraction type,Music,Duration,Simplified Chinese,Traditional Chinese,Hanyu Pinyin,Literal meaning,Transcriptions,Bopomofo,Gwoyeu Romatzyh,Wade–Giles,IPA,Yale Romanization,Jyutping,Hokkien POJ,Animation by,Color process,Engine(s),Genre(s),Actor control,Production company,Release(s),Format(s),Simplified,Characters,Date premiered,Place premiered,Setting,Original language,Subject,Published,Dewey Decimal,Text,Illustrator,Original title,Published in English,French,Developed by,Ending theme,No. of seasons,Nationality,Portrayed by,Alias,Species,Gender,Family,Alma mater,Camera setup,Novel(s),Comics,Film(s),Screen story by,Hangul,Revised Romanization,McCune–Reischauer,Developer(s),Publisher(s),Designer(s),Programmer(s),Artist(s),Writer(s),Engine,Platform(s),Release,Mode(s),Original work,Television series,Japanese,Hepburn,Literally,Cantonese,Full name,Height,Seasons,Chinese,Other names,Relatives,Yiddish,Formerly,Key people,Total assets,Owner,Number of employees,Divisions,Subsidiaries,Arabic,Romanized,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay","[Cliff Eidelman, Yello]",Oliver Wood,Michael Tronick,Silver Pictures,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,English,$20 million,$21.4 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Maurice Jarre,Mark Plummer,Howard E. Smith,Avenue Pictures,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,English,$6 million,$2.7 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [8]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
# 11. Check the ratings DataFrame.
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
